Firstly, enable the GPU within the notebook settings.

1. Download the training data from the following link : https://drive.google.com/open?id=10niBg6RVnK5DUgwdrtBoAjgj309yZ4Q7
When uploading the file, it may need to be renamed to (dataset.zip)
2. Download the latest model weights from : https://drive.google.com/open?id=1DBeBsktlh5Pp1w21QWrJ7-fC1wsXbhAg

Run the following cell to clone the needed repositories and set up the model for training alongside creating the needed files for training the model. 

In [0]:
# Clones the github containing the darknet model
!git clone https://github.com/AlexeyAB/darknet/

# Clones the github used to convert the model into a keras format
!git clone https://github.com/qqwweee/keras-yolo3

# Goes into the make file and allows the model to use CUDNN and train using a 
# GPU
%cd /content/darknet
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile

# Makes the model

%cd /content/darknet
!make

# Unzip the dataset
%cd /content/
!unzip /content/dataset.zip

# Create the train.txt file used during training to tell the model where to 
# find the training data. A test.txt file is not used as the model is not 
# evaluated here. The function iterates through all of the training data and 
# write the filenames into a text file.
import os 
def create_train_file():
  directory = "/content/dataset"
  counter = 0
  files= []
  try:
    for filename in os.listdir(directory):
      if filename.endswith('.jpg'):
        counter+=1
        files.append(os.path.join('/content/darknet/dataset/'+filename))
    file_new = open('/content/train.txt','w')
    for i in range(len(files)):
      file_new.write(files[i]+"\n")
  except FileNotFound:
    print("Dataset folder could not be located")

# Calls the function twice because of a bug in Colab when writing files
for i in range(2):
  create_train_file()


# Moves the dataset file into the darkent directory
!mv /content/dataset /content/darknet

# Moves the train.txt file into darket
!mv /content/train.txt /content/darknet

# Calls the function twice because of a bug in Colab when writing files
# Creates the obj.data file needed for training the model
for i in range(2):
  obj_data = open('/content/obj.data','w')
  obj_data.write("classes= 2"+"\n"+"train  = train.txt" +"\n"+"train  = train.txt" +"\n"+
          "names = obj.names "+"\n"+"backup = backup/")

# Moves the created files into the correct directories for the model to locate
!mv /content/darknet/dataset/classes.txt /content/darknet/cfg
!mv /content/obj.data /content/darknet/cfg
!mv /content/yolov3-tiny_last.weights /content/darknet

# Renames the classes file to the expected filename
os.rename('/content/darknet/cfg/classes.txt','/content/darknet/cfg/obj.names')


Cloning into 'darknet'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 13029 (delta 0), reused 1 (delta 0), pack-reused 13025
Receiving objects: 100% (13029/13029), 11.76 MiB | 14.12 MiB/s, done.
Resolving deltas: 100% (8900/8900), done.
Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 998.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/darknet
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_open

Call this cell to locate the anchors needed for the system to detect the hand within the images

In [0]:
%%bash
cd darknet
./darknet detector calc_anchors /content/darknet/cfg/obj.data -num_of_clusters 6 -width 416 -height 416 -show


 num_of_clusters = 6, width = 416, height = 416 
 read labels from 1190 images 
 loaded 	 image: 1190 	 box: 1715
 all loaded. 

 calculating k-means++ ...

 iterations = 49 


 avg IoU = 81.14 % 

Saving anchors to the file: anchors.txt 
anchors =  42,102,  56,102,  82,200, 113,161, 104,252, 149,304


 CUDA-version: 10000 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
Unable to init server: Could not connect: Connection refused

(clusters:1387): Gtk-WARNING **: 16:14:37.223: cannot open display: 


This cell reads the anchors previously created and ammends them within the default tiny yolo config file. The corrects are then made to the number of classes to detect alongside the number of filters calculated as (number of classes to detect + 5 (the four bounding box co-ordinates plus the confidence of the prediction) x 3 (number of channels)).

The file is then changed back into cfg format 

In [0]:
try:
  anchors = open('/content/darknet/anchors.txt','r')
except FileNotFoundError:
  print("Anchors could not be found")

contents =anchors.read()
try:
  os.rename('/content/darknet/cfg/yolov3-tiny.cfg','/content/darknet/cfg/yolov3-tiny.txt')
except FileNotFoundError:
  print("Tiny config could not be located")

try:
  tiny_yolo_config = open('/content/darknet/cfg/yolov3-tiny.txt','r+')
except FileNotFoundError:
  print("Conversion file could not be located")

content = tiny_yolo_config.readlines()
content[133] = "anchors = "+ contents +"\n"
content[134] = "classes=3\n"
content[176] = "classes=3\n"
content[175] = "anchors = "+ contents +"\n"
content[170] = "filters= 24\n"
content[126] = "filters= 24\n"
content[19] = "max_batches = 600000\n"
content[21] = "steps=4000,5400\n"
content[6] = "# subdivisions=2\n"
# Changes the saturation and exposure within the image during training.
content[13] = "saturation = 1.5\n"
content[14] = "exposure = 1.5\n"
string = ""
for line in content:
  string+=line
tiny_yolo_config.seek(0)
tiny_yolo_config.write(string)

try:
  os.rename('/content/darknet/cfg/yolov3-tiny.txt','/content/darknet/cfg/yolov3-tiny.cfg')
except FileNotFound:
  print("File could not be located for conversion")

This cell is used to train the model. The weights are used from the last batch during previous training. The output new weights are located in the darknet/backup file.

In [0]:
%%bash
cd darknet
./darknet detector train cfg/obj.data cfg/yolov3-tiny.cfg /content/darknet/yolov3-tiny_last.weights -dont_show 0

Process is interrupted.


The new weights are added to the config file and converted into Keras format using the convert.py python script by qqwweee.

Download the yolo.h5 file and place it within the same directory as the python scripts.

In [0]:
%%bash
cd /content/keras-yolo3/
python convert.py /content/darknet/cfg/yolov3-tiny.cfg /content/darknet/backup/yolov3-tiny_last.weights /content/yolo.h5

Loading weights.
Weights Header:  0 2 5 [515800]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 16)
Parsing section maxpool_0
Parsing section convolutional_1
conv2d bn leaky (3, 3, 16, 32)
Parsing section maxpool_1
Parsing section convolutional_2
conv2d bn leaky (3, 3, 32, 64)
Parsing section maxpool_2
Parsing section convolutional_3
conv2d bn leaky (3, 3, 64, 128)
Parsing section maxpool_3
Parsing section convolutional_4
conv2d bn leaky (3, 3, 128, 256)
Parsing section maxpool_4
Parsing section convolutional_5
conv2d bn leaky (3, 3, 256, 512)
Parsing section maxpool_5
Parsing section convolutional_6
conv2d bn leaky (3, 3, 512, 1024)
Parsing section convolutional_7
conv2d bn leaky (1, 1, 1024, 256)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 256, 512)
Parsing section convolutional_9
conv2d    linear (1, 1, 512, 24)
Parsing section yolo_0
Parsing section route_0
Parsing section convolutional_10
con

Using TensorFlow backend.






2020-04-02 18:31:15.950211: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-04-02 18:31:15.955542: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000165000 Hz
2020-04-02 18:31:15.955816: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1348bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-02 18:31:15.955847: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-04-02 18:31:15.973766: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-02 18:31:16.134644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA nod